# Thermal Preference Elicitation Framework

Currently, this framework has been validated for only 1D (operating temp) feature.

# Step 1

Load required modules

In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../')
from GPElicit import elicit
from Misc import visualize

/Users/nimishawalgaonkar/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Step 2

1. Occupant walks inside the room and is exposed to state r.
2. Randomly change the state of the room to new state s.
3. Ask the occupant which state they prefer. 
4. If they say state current state s, then record y as 1. If they say previous state r, then record y as 0.
This first duel is our initial duel.
5. Put the duel value and response in the '../data/duels/duels.csv' file


In [2]:
# Load the duels file
data_file = '../data/duels/duels1.csv' # <--- you need to keep on updating this file

In [3]:
data = pd.read_csv(data_file)
data # check if the duels data is as you want it to be

,Unnamed: 0,Tprev,Tcurrent,y,MEUI
0,0,15,20,1,NaN


In [4]:
iter_num = data.shape[0]

In [5]:
Tprev = np.array(data.Tprev) # previous state (operating temp.)
Tcurrent = np.array(data.Tcurrent) # current state <---- elicitation framework will give you this values
X = np.vstack([[Tprev, Tcurrent]]).T
X = X.astype(float) # features need to be float
Y = np.array(data.y)[:,None] # response of the occupant <---- you need to ask occupant about this

# Step 3

Now, we want to find the next elicited state. So, we need to use **elicit** module.

In [6]:
config_file = '../config_files/thermal_config.json' # configuration for grid (how fine you want to be)
trial_num = 1 # this is just to save all the plots in '../data/results/T1' if trial_num = 1 ; T2 if trial_num = 2

In [ ]:
next_state, next_duel, meanexp, max_exp_imp = elicit.Aquisition(X,Y, config_file, 2).EUI(iter_num,
                                                                                     trial_num, savefig = True)

## Step 3 is the most important step. It outputs next state.

# next_state ?

In [ ]:
next_state

# next duel ?

One of the state is always shared between two duels. So, this is nothing but concatenation of next_state with previous.

In [ ]:
next_duel

# max EUI ?

In [ ]:
max_exp_imp

# Step 4 (Sanity checks)

Is our framework on the right track?

Check -
1. Max Expected Improvement value (is it less than the previous iteration's expected improvement? If so, GOOD!
2. Check the Expected Improvement plots. This will be saved automatically in '../data/results/T1/exp_imp_plots/iteration_num'.
3. Also check utility samples, how they look? Do they make sense? Is our framework going towards max? Some utility samples will also be saved in '../data/results/T1/utility_samples/iteration_num'.

# Step 5

1. Once you have verified that the framework is on the right track, change the operating temp of the room to the next_state as above. 
2. Its fine if you are not able to acheive the next state accurately, just record the measured next state value.
2. Ask the occupant again, which state does he prefer.
3. Add the new measured state and response to the csv  '../data/duels/duels.csv' file.
4. Add max_exp_imp to the MEUI column.
5. Run the notebook again with updated csv file.
6. As you progress with the elicitation, you will notice that the ratio $(MEUI_{(i+1)} - MEUI_{(i)})/MEUI_{(i)}$ will decrease. Based on pilot study, stop the elicitation, once the ratio becomes small enough.